## Call Libraries

In [2]:
import pandas as pd
import numpy as np
import datetime

In [3]:
# for visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [5]:
# for the dashboard
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State

## Call the Data

In [6]:
# Path of the data's location
path = 'data/'

In [7]:
# Call the data
df = pd.read_csv(path + 'covid_vaccine.csv')
df.head()

,state,date,confirmed,deaths,population,fully,at_least_one
0,Alabama,2020-01-22,0,0,NaN,NaN,NaN
1,Alabama,2020-01-23,0,0,NaN,NaN,NaN
2,Alabama,2020-01-24,0,0,NaN,NaN,NaN
3,Alabama,2020-01-25,0,0,NaN,NaN,NaN
4,Alabama,2020-01-26,0,0,NaN,NaN,NaN


## Button Setting

In [8]:
button = ['confirmed', 'deaths']
states = df['state'].unique()

## App & Layout

In [9]:
# App structure
app = dash.Dash(__name__)
app.title = ('Dashboard | COVID-19 & Vaccines')
server = app.server

# App layout
app.layout = html.Div([
    # Title
    html.Div([html.H2('Trends in COVID-19 Cases and Vaccination in The United States', style={'textAlign': 'center'})]),
    
    # Dropdown - States
    html.Div([dcc.Dropdown(id = 'id_state',options=[{'label':i, 'value':i} for i in states],value = states[0])], style={'width':'200px'}),
    html.Br(),
    
    # Button - Confirmed, Deaths
    html.Div([dcc.RadioItems(id='id_covid', options=[{'label': i, 'value': i} for i in button], value = button[0])]),
    html.Br(),
    
    # Chart
    html.Div([dcc.Graph(id = 'graph', style={'height':650})]
            , style={'width': '95%', 'margin-left': 'auto', 'margin-right': 0})
])

## Graphs

In [10]:
@app.callback(Output('graph', 'figure'), 
              [Input('id_state', 'value'),
               Input('id_covid', 'value')])

# Define by input order = (1) state, (2) covid
def update_graph(stat, cov):
    
    # secondary y-axis setting,
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    
    # text - legend name & yaxis label
    if cov == 'confirmed':
        legend = 'Confirmed'
        ytitle = '<b>Number of Confirmed Cases</b>'
    else:
        legend = 'Deaths'
        ytitle = '<b>Number of Deaths Cases</b>'
    
    
    # Selecting the data
    df1 = df[df['state'] == stat]
    
    # primary Y : Covid-19
    fig.add_trace(go.Bar(x = df1['date'],
                         y = df1[cov],
                         name = legend,
                         hovertemplate = '%{y:,}',
                         marker_color='rgb(185,185,185)'),
                  secondary_y=False)
        
    fig.update_layout(go.Layout(xaxis = dict(title = 'Time (day)'),
                                yaxis = dict(title = ytitle, showgrid = False),
                                legend = dict(orientation='h', yanchor='top', y=1.1, traceorder='normal'),
                                hovermode = 'x unified',
                                plot_bgcolor='white'))
    
    
    # secondary Y : Fully Vaccine %
    fig.add_trace(go.Scatter(x = df1['date'],
                             y = df1['fully']/df1['population']*100,
                             name = 'Fully',
                             hovertemplate='%{y:,.1f}%',   # one decimal place
                             mode="lines",
                             line={'width':2.5}),
                  secondary_y=True)
    
    # secondary Y : At least one Vaccine %
    fig.add_trace(go.Scatter(x = df1['date'],
                             y = df1['at_least_one']/df1['population']*100,
                             name = 'At least 1',
                             hovertemplate='%{y:,.1f}%',   # one decimal place
                             mode="lines",
                             line={'width':2.5}),
                  secondary_y=True)
    
    # secondary_y axis : title, suffix
    fig.update_yaxes(title_text='<b>Series Complete (% of Population)</b>', 
                     ticksuffix = '%', 
                     range = [0, 100],
                     secondary_y=True)

    return fig

In [ ]:
# Run App
if __name__=='__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Aug/2022 14:43:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 14:43:21] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_4_1m1653418901.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 14:43:21] "GET /_dash-component-suites/dash/deps/react@16.v2_4_1m1653418901.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 14:43:21] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_4_1m1653418901.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 14:43:21] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_4_1m1653418901.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 14:43:21] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_4_1m1653418901.min.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 14:43:21] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_4_0m1653418901.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 14:43:21] "GET /_dash-component-suites/d